In [1]:
## associate channels and filter relevant channels for SWR and spindles

# LOAD PACKAGES

from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path

# LOAD fILES CA1
folder_input = Path('Gaelle/Baseline_recording/BlueLines/session3/OpenEphys/')
EMGbooleaninput = folder_input / f'EMGframeBoolean.pkl'
EMGboolean = pd.read_pickle(EMGbooleaninput)
LFPwakeremovedinput = folder_input / f'LFPwake0.npy'
LFPwakeremoved = np.load(LFPwakeremovedinput, mmap_mode= 'r')
# For long recordings, use LFPwakeremoved directly to nt bother with super long wake periods during assessment
Allinput = folder_input / f'LFPwake0.npy'
All = np.load(Allinput, mmap_mode= 'r')

#######################################################################
#######################################################################
#                           ### Spindles ###                          #
#######################################################################
#######################################################################


# LOAD fILES PFC, S1
S1 = All[:, 18]-All[:, 19]
S1wakeremoved = LFPwakeremoved[:,18]-LFPwakeremoved[:,19]

## BAND PASS FILTER - 8-20 Hz -
# Filtre parameter:
f_lowcut = 8.
f_hicut = 20.
fs = 1000
nyq = 0.5 * fs
N = 4                 # Filtre order
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filtering:
b, a = signal.butter(N, Wn, 'band')


filt_S1 = signal.filtfilt(b, a, S1)
filt_S1wakeremoved = signal.filtfilt(b, a, S1wakeremoved)

## Continuous Wavelet Transform

# Parameter and computation of CWT
w = 8.
freq = np.linspace(10, 18, 8)
widths = w*fs / (2*freq*np.pi)
S1NWcwt = signal.cwt(filt_S1wakeremoved, signal.morlet2, widths, w=w)

# Projection calculation S1
absS1NWcwt = np.absolute(S1NWcwt)
proj_S1NWcwt = np.sum(absS1NWcwt, axis = 0)/10
sdproj_S1cwt = np.std(proj_S1NWcwt)
sd7proj_S1cwt = sdproj_S1cwt*3

from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths
#####################################
########         S1         #########
#####################################
cortex = S1
proj_cortex = proj_S1NWcwt #proj_S1W0Lcwt
proj_cortexC = proj_S1NWcwt #proj_S1W0Ccwt
sd_proj_cortex = sd7proj_S1cwt
lib_wake0_cortex = S1NWcwt # S1cwtWake0lib
filt_cortex = filt_S1

# 7 sd threshold
peaks, properties = find_peaks(proj_cortex, prominence=1, width=200, height=sd_proj_cortex)

# Spindles boundaries taken at 70% from peak of intensity. This means that the spindles with small amplitude will be longer than the big ones.
results_width = peak_widths(proj_cortex, peaks, rel_height=0.6)

# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
Spindle_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

projMaxP_cwtmg = np.max(lib_wake0_cortex, axis = 0)
projMaxF_cwtmg = np.argmax(lib_wake0_cortex, axis = 0)/2 + 10

nb_Spindles = np.arange(0,len(peaks),1)
data = np.zeros((len(peaks),4))

for tt in nb_Spindles:
    Spindle_start = int(Spindle_prop[3,tt])
    Spindle_stop = int(Spindle_prop[4,tt])
    Spindle_MaxP = projMaxP_cwtmg[Spindle_start:Spindle_stop]
    Spindle_MaxF = projMaxF_cwtmg[Spindle_start:Spindle_stop]
    data[tt, 0] = max(Spindle_MaxF).round()
    data[tt, 1] = max(Spindle_MaxP).round()
    data[tt, 2] = round(sum(Spindle_MaxF)/len(Spindle_MaxF))
    data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))

param_Spindle = pd.DataFrame(data, columns = ['Max freq', 'Max int', 'Avg freq', 'Avg int'])
tSpindle_prop = Spindle_prop.transpose()
pd_prop_Spindle = pd.DataFrame(tSpindle_prop, columns = ['peak time', 'Duration', 'peak amp', 'start time', 'end time'])
All_Spindle = pd.concat([pd_prop_Spindle, param_Spindle], axis=1)

folder_output = Path('/crnldata/waking/audrey_hay/')
filename2 = folder_input / f'Spindleproperties_S1.pkl'
filename3 = folder_input / f'Spindleproperties_S1.csv'
All_Spindle.to_pickle(filename2)
All_Spindle.to_csv(filename3, sep = ',')

combined = np.stack([filt_S1wakeremoved, proj_S1NWcwt], axis = 1)
filenameC = folder_input / f'SignalS1.npy'
np.save(filenameC, combined)

/var/folders/jq/nz6nvbd54ns2lkjvj31_7_4r0000gn/T/ipykernel_46359/1588084260.py:99: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 1] = max(Spindle_MaxP).round()
/var/folders/jq/nz6nvbd54ns2lkjvj31_7_4r0000gn/T/ipykernel_46359/1588084260.py:101: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))
/var/folders/jq/nz6nvbd54ns2lkjvj31_7_4r0000gn/T/ipykernel_46359/1588084260.py:101: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))
